# Find similar pictures in a directory
This notebooks shows how to find similar pictures in a directory using a pre-trained deep learning model that transforms
images into embeddings. The embeddings are then used to find similar images in the directory.

In [5]:
import os
import sys

# Add src directory to the Python path
sys.path.append(os.path.abspath('../src'))

from PIL import Image
import pandas as pd

from src.ImageEmbedder import ImageEmbedder
from src.EmbeddingRetriever import EmbeddingRetriever
from IPython.display import display, Image

ModuleNotFoundError: No module named 'ImageEmbedder'

# Create image embeddings

In [ ]:
path = "C:/Users/User/Desktop/toprocess"
ie = ImageEmbedder()

In [3]:
# List all image files in the folder based on allowed extensions
def list_images_with_full_path(folder_path, allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']):
    image_files = [os.path.join(os.path.abspath(folder_path), f) 
                   for f in os.listdir(folder_path) 
                   if os.path.splitext(f)[1].lower() in allowed_extensions]
    return image_files

img_paths = list_images_with_full_path(path)

In [ ]:
# create and store all embeddings
for img_path in img_paths:
    ie.get_embedding(img_path)
ie.save_embeddings("data/embeddings.parquet")

# Find similar images

In [5]:
embeddings_df = pd.read_parquet("data/embeddings.parquet")
retriever = EmbeddingRetriever(embeddings_df)

In [ ]:
df_similar_images = retriever.update_similar_images(output_file='data/similar_images.parquet')
df_similar_imagesb

In [ ]:
# find similar images to a given image

def display_similar_images(path):
    display(Image(filename=path, width=200))
    print(f"Image: {path}")

    similar_images = retriever.find_similar_embeddings(path)
    for fpath, sim, embedding in similar_images:
        display(Image(filename=fpath, width=200))
        print(f"Similarity: {sim:.2f}, Embedding: {embedding[:5]}...")


# Call the function to display similar images
image_path = "image_path"
display_similar_images(image_path)